# CS6140 Assignments

**Instructions**
1. In each assignment cell, look for the block:
 ```
  #BEGIN YOUR CODE
  raise NotImplementedError.new()
  #END YOUR CODE
 ```
1. Replace this block with your solution.
1. Test your solution by running the cells following your block (indicated by ##TEST##)
1. Click the "Validate" button above to validate the work.

**Notes**
* You may add other cells and functions as needed
* Keep all code in the same notebook
* In order to receive credit, code must "Validate" on the JupyterHub server

---

# Assignment 4: Linear Models (2)


In this exercise, we will transform our gradient descent algorithm into stochastic gradent descent. We will then implement linear regression and logistic regression. Finally, we will run these models on a real-world dataset.

In [1]:
require './assignment_lib'

"if(window['d3'] === undefined ||\n   window['Nyaplot'] === undefined){\n    var path = {\"d3\":\"https://cdnjs.cloudflare.com/ajax/libs/d3/3.5.5/d3.min\",\"downloadable\":\"http://cdn.rawgit.com/domitry/d3-downloadable/master/d3-downloadable\"};\n\n\n\n    var shim = {\"d3\":{\"exports\":\"d3\"},\"downloadable\":{\"exports\":\"downloadable\"}};\n\n    require.config({paths: path, shim:shim});\n\n\nrequire(['d3'], function(d3){window['d3']=d3;console.log('finished loading d3');require(['downloadable'], function(downloadable){window['downloadable']=downloadable;console.log('finished loading downloadable');\n\n\tvar script = d3.select(\"head\")\n\t    .append(\"script\")\n\t    .attr(\"src\", \"http://cdn.rawgit.com/domitry/Nyaplotjs/master/release/nyaplot.js\")\n\t    .attr(\"async\", true);\n\n\tscript[0][0].onload = script[0][0].onreadystatechange = function(){\n\n\n\t    var event = document.createEvent(\"HTMLEvents\");\n\t    event.initEvent(\"load_nyaplot\",false,false);\n\t    win

true

## Question 1.1 (10 points)

Transform your batch gradient descent into a stochastic gradient descent algorithm. Using a class here allows the SGD algorithm to maintain state, such as learning rate and weights. Test your algorithm on the coin dataset and Binomial model you coded in Assignment 3. Plot the likelihood measured for each batch of 100 examples on one pass of the dataset.

We will implement mini-batch SGD. Therefore you will not have to alter your Binomial Model. 

Stochasic gradient descent requires a learning rate that decreases with each iteration. Use the following learning rate:

## $\eta = \frac{\eta_{0}}{\sqrt{t}}$

where $\eta_{0}$ is the initial learning rate and $t$ is the number of mini-batch iterations.

The ```update``` function updates the member variable ```@weights``` and potentially other parameters based on the dataset. For SGD, now ```x``` is not the full dataset but a batch of examples. It needs to call the ```grad``` function in the objective, and there is no return value.


Example lines with ```__note__``` indicate placeholders for code:

```ruby
def update x
    @objective.grad(x, @weights)
    learning_rate = __calculate_learning_rate__
    __update_each_weight__ #based on the update formula and the learning rate
    __don_t_forget_to_call_adjust__
end
```

In [2]:
class StochasticGradientDescent
  attr_reader :weights
  attr_reader :objective
  def initialize obj, w_0, lr = 0.01
    @objective = obj
    @weights = w_0
    @n = 1.0
    @lr = lr
  end
  def update x
    # BEGIN YOUR CODE

    lr  = @lr.to_f / Math.sqrt(@n)
    @n +=1

    i = 0
    gradient = @objective.grad x,@weights
    while i < @weights.length do
      if @weights[@weights.keys[i]].is_a?Numeric and gradient[@weights.keys[i]].is_a?Numeric
        @weights[@weights.keys[i]]  = @weights[@weights.keys[i]] - lr*gradient[@weights.keys[i]]
      end
      i+=1
    end    
    
   @objective.adjust @weights
    
    #END YOUR CODE
  end
end

:update

In [3]:
### TEST ###
# Demonstrates that weights are stored in the gradient function. 
# This is included in the provided code, so just an example of how to use SGD
class EmptyObjective
  def func x, w; end
  def grad x, w; end
  def adjust w; end
end

t0_w = {"0" => 1.23, "1" => 4.56}
t0_obj = EmptyObjective.new
t0_sgd = StochasticGradientDescent.new t0_obj, t0_w, 0.25

puts t0_sgd.weights
assert_in_delta 1.23, t0_sgd.weights["0"], 1e-2

{"0"=>1.23, "1"=>4.56}


In [4]:
### TEST ###
# Perform a single update to the weights and see that the objective function value is different.
# This is a no-data objective, so we are not providing any data, so our batch of examples is empty.
class ParabolaObjective
  def func x, w
    0.5 * ((w["x0"] - 1) ** 2.0 + (w["z1"] - 2) ** 2.0)
  end
  def grad x, w
    dw = {"x0" => (w["x0"] - 1), "z1" => (w["z1"] - 2)}
  end
  def adjust w
  end
end

def test_1_1_0
  w = {"x0" => 0.0, "z1" => 0.0}
  learning_rate = 0.25
  obj = ParabolaObjective.new
  sgd = StochasticGradientDescent.new obj, w, learning_rate
  batch_of_examples = []
  
  lik_0 = obj.func(batch_of_examples, sgd.weights)
  puts lik_0
  
  #Calculate objective for initial weights
  assert_in_delta 0.0, sgd.weights["x0"], 1e-2
  assert_in_delta 2.5, lik_0, 1e-2

  #Update weights
  sgd.update(batch_of_examples)
  puts sgd.weights
  
  #Now, objective function is different for the newly updated weights
  lik_1 = obj.func(batch_of_examples, sgd.weights)
  puts lik_1
  
  assert_in_delta 0.25, sgd.weights["x0"], 1e-2
  assert_in_delta 1.40625, lik_1, 1e-2
end

test_1_1_0()

2.5
{"x0"=>0.25, "z1"=>0.5}
1.40625


In [5]:
### TEST ###
# Testing on a known objective. Repeat SGD for 1000 iterations and expect to converge to correct solution.

def test_1_1_1
  w = {"x0" => 0.0, "z1" => 0.0}
  learning_rate = 0.25
  obj = ParabolaObjective.new
  sgd = StochasticGradientDescent.new obj, w, learning_rate
  batch_of_examples = []

  lik = 1.0
  1000.times do 
    sgd.update(batch_of_examples)
    lik = obj.func(batch_of_examples, sgd.weights)
  end

  assert_true(lik < 0.1, "SGD converges with simple objective")
  assert_in_delta 1.0, sgd.weights["x0"], 0.1, "Weight 0 expected to be 1.0"
  assert_in_delta 2.0, sgd.weights["z1"], 0.1, "Weight 1 expected to be 2.0"
end

test_1_1_1()

## Begin Question 2.1 (10 points)

Implement linear regression as an objective function for use with stochastic gradient descent. First, we will implement the predict function. For a weight vector, $w$ and a single ```Row``` with features, $x$, implement:

### $f(w,x) = w^T x$


Note that in the formula above ```x``` is the ```features``` key in the example. Check out the ```fetch``` and ```has_key?``` methods in ruby.

In [6]:
class LinearRegressionModel  
  def predict example, weights
    # BEGIN YOUR CODE
    sum = 0.0 
    i = 0
    name  = example["features"].keys
    while i< weights.length do
      n = weights.keys[i] 
      if name.include?n 
        if example["features"][n].is_a?Numeric and weights[n].is_a?Numeric
          sum += example["features"][n]*weights[n]
        else
          sum += 0.0
        end
      end
      i+=1
    end
    
 
    return sum
    
    
    #END YOUR CODE
  end
end

:predict

In [7]:
### TEST ###
# Tests that function can predict when all keys are found both vectors
def test_2_1_0
  lr = LinearRegressionModel.new
  example = {"features" => {"a" => 2.0}}
  weights = {"a" => 3.0}
  
  assert_in_delta 6.0, lr.predict(example, weights), 1e-6
end


:test_2_1_0

In [8]:
### TEST ###
# Tests that function can predict even when weights are missing
def test_2_1_1
  lr = LinearRegressionModel.new
  
  puts "Handles empty weights"
  empty_example = {"features" => {}}
  empty_weights = {}  
  assert_equal 0.0, lr.predict(empty_example, empty_weights)
  
  puts "Handles weights not in features"
  weights_without_a = {"a" => 0.0, "b" => 1.0}
  example_a = {"features" => {"a" => 1.0}}
  assert_equal 0.0, lr.predict(example_a, weights_without_a)
  
  puts "Handles features not in weights"
  example_bc = {"features" => {"b" => 7.2, "c" => 2.5}}
  assert_in_delta 7.2, lr.predict(example_bc, weights_without_a), 1e-2
end

test_2_1_1()

Handles empty weights
Handles weights not in features
Handles features not in weights


## Begin Question 2.2 (10 points)

Continuing the implementation, implement the $L_2$ loss, which applies to a mini-batch of $n$ points. Use the ```predict``` function you implemented earlier.

### $L(w,X) = \frac{1}{n} \sum_{i} \frac{1}{2} \left(f(w,x_i) - y_i\right) ^ 2$

where $x_i$ is the ```features``` of the example, $y_i$ is the ```label``` of the example, and $n$ is the number of examples in the batch.

In [9]:
class LinearRegressionModel
  def func examples, w
    # BEGIN YOUR CODE
    sum  = 0.0
    i = 0
    obj = LinearRegressionModel.new
    adjust w
    while i< examples.length do 
      if examples[i]["label"].is_a?Numeric
        sum += (obj.predict(examples[i], w) -  examples[i]["label"]) **2
      else
        sum += 0.0
      end
      i+=1
    end
    
    return sum/(2.0*examples.length)
    
    #END YOUR CODE
  end
  ## Adjusts the parameter to be within the allowable range
  def adjust w
    
  end
end

:adjust

In [10]:
### TEST ###
# Use the coin dataset to measure the loss for a single-example batch.
def test_2_2_0
  #Use a fixed seed so that the coin dataset returns the same value each time
  dataset = coin_dataset(100)
  examples = dataset["data"]
  assert_equal 100, examples.size
  
  # Batch of size 1 from a dataset of 100 examples
  batch = examples[0,1]
  puts batch  
  assert_equal 1, batch.size
  
  #Start with mu parameter 0.1, denoted by bias
  model = LinearRegressionModel.new
  weights_mu_0_1 = {"bias" => 0.5}
  
  #Calculate the squared loss
  f = model.func batch, weights_mu_0_1
  puts f
  assert_in_delta 0.125, f, 0.050, "Expected loss within [250, 350]"
end

test_2_2_0()


[{"features"=>{"bias"=>1.0}, "label"=>1.0}]
0.125


In [11]:
### TEST ###
# Use the coin dataset
t22_data = coin_dataset(1000)

t22_model = LinearRegressionModel.new
t22_w = Hash.new
t22_w["bias"] = 0.1

t22_f = t22_model.func t22_data["data"], t22_w
assert_in_delta 0.300, t22_f, 0.050, "Expected loss within [250, 350]"

t22_w["bias"] = 0.77
t22_f = t22_model.func t22_data["data"], t22_w
assert_in_delta 0.090, t22_f, 0.050, "Expected loss for a closer guess to be within [40, 140]"

## Begin Question 2.3 (10 points)

Continuing the implementation of linear regression, now implement the gradient function. This returns a gradient vector for the mini-batch of $n$ points.

Note that the gradient for a batch is the average gradient for all examples in the batch, which is why there is an initializer for the ```gradient``` hash.

In [12]:
class LinearRegressionModel
  def grad examples, weights
    gradient = Hash.new {|h,k| h[k] = 0.0}
    # BEGIN YOUR CODE

    sum = 0.0
    i=0
    model = LinearRegressionModel.new
    while i<examples.length do
      sum += model.predict(examples[i], weights)  - examples[i]["label"]
      i+=1
    end
    gradient.store("bias", sum/examples.length)
    
    
    #END YOUR CODE
    return gradient
  end
end

:grad

In [13]:
### TEST ###
# Calculate the gradient update on a batch of 5 examples

def test_2_3_0 
  #Use a fixed seed so the gradient value comes out the same
  srand 1295187568912756074856
  dataset = coin_dataset(1000)
  model = LinearRegressionModel.new
  
  #Batch of size 5 of 1000
  examples = dataset["data"]
  batch = examples[0,5]
  puts "Batch:"  
  puts batch
  assert_equal 5, batch.size
  
  #Initialize weights with mu value represented by bias
  weights = Hash.new
  weights["bias"] = 0.1

  gradient = model.grad batch, weights
  puts "Gradient:"
  puts gradient
  assert_in_delta -0.5, gradient["bias"], 0.2, "Expected loss within [-0.49, -0.89]"
end

test_2_3_0()

Batch:
[{"features"=>{"bias"=>1.0}, "label"=>1.0}, {"features"=>{"bias"=>1.0}, "label"=>0.0}, {"features"=>{"bias"=>1.0}, "label"=>1.0}, {"features"=>{"bias"=>1.0}, "label"=>1.0}, {"features"=>{"bias"=>1.0}, "label"=>0.0}]
Gradient:
{"bias"=>-0.5}


In [14]:
### TEST ###

t23_data = coin_dataset(1000)
t23_model = LinearRegressionModel.new
t23_w = Hash.new
t23_w["bias"] = 0.1

t23_g = t23_model.grad t23_data["data"], t23_w
assert_in_delta -0.69, t23_g["bias"], 0.2, "Expected loss within [-0.49, -0.89]"

t23_w["bias"] = 0.77
t23_g = t23_model.grad t23_data["data"], t23_w
assert_in_delta 0.0, t23_g["bias"], 0.1, "Expected loss for a better guess to be within [-0.1, 0.1]"

## Question 2.4 (10 points)

Putting the previous steps together, use your ```StochasticGradientDecent``` to run linear regression for 10 passes (epochs) over the Coin Dataset, each pass with a mini-batch of size 20. Tune the learning rate, ```lr```, so that the model converges well. Assume that ```obj``` is an instance of ```LinearRegressionModel```, and ```w``` is an initial weight vector.

Track the number of batches in the ```iters``` array and the loss in the ```losses``` array.

In [15]:
def train_coin_sgd(obj, w, dataset)
  i = 0
  iters = []
  losses = []
  
  #Define sgd = StochasticGradientDescent.new obj, w, lr
  # You set the learning rate, lr
  # BEGIN YOUR CODE

    lr = 3.7
    sgd= StochasticGradientDescent.new( obj, w, lr)
    10.times do 
      iters.append(i)
      losses.append(obj.func(dataset["data"][0,20],w))
      sgd.update(dataset["data"][0,20])
      w = sgd.weights

      i+=1  
    end

  
  
  
  #END YOUR CODE
  return [sgd, iters, losses]
end

:train_coin_sgd

In [16]:
### TEST ###
t24_data = coin_dataset(1000)
t24_model = LinearRegressionModel.new
t24_w = Hash.new
t24_w["bias"] = 0.1

t24_trainer, t24_iter, t24_losses = train_coin_sgd t24_model, t24_w, t24_data

assert_true t24_w.has_key?("bias")
assert_in_delta 0.77, t24_w["bias"], 0.1, "Expected weight for 'bias'  [0.67, 0.87]"
t24_cum_loss = 0.0
t24_losses.each_index {|i| t24_cum_loss += t24_losses[i]; t24_losses[i] = t24_cum_loss / (t24_iter[i] + 1)}
Daru::DataFrame.new({x: t24_iter, y: t24_losses}).plot(type: :line, x: :x, y: :y) do |plot, diagram|
  plot.x_label "Batches"
  plot.y_label "Cumulative Loss"
end

#<Nyaplot::Plot:0x000055ec45041d98 @properties={:diagrams=>[#<Nyaplot::Diagram:0x000055ec450249a0 @properties={:type=>:line, :options=>{:x=>:x, :y=>:y}, :data=>"3781e45e-1357-4265-a2a6-d212e2fc45be"}, @xrange=[0, 9], @yrange=[0.34500000000000014, 3.983397966303145]>], :options=>{:x_label=>"Batches", :y_label=>"Cumulative Loss", :zoom=>true, :width=>700, :xrange=>[0, 9], :yrange=>[0.34500000000000014, 3.983397966303145]}}>

## Question 3.1 (10 points)

Implement Logistic Regression, following much the same process as with linear regression. The prediction function returns a value:

### $f(x,w) = \frac {1}{1 + \exp \left( -w^T x \right) } $

In [17]:
class LogisticRegressionModel
  def predict row, w
    # BEGIN YOUR CODE
    adjust w
    sum = 0.0 
    i = 0
    name  = row["features"].keys

    while i< w.length do
      n = w.keys[i] 
      if name.include?n 
        if row["features"][n].is_a?Numeric and w[n].is_a?Numeric
          sum += row["features"][n]*w[n]
        else
          sum += 0.0
        end
      end
      i+=1
    end
   
       
    return 1.0/(1.0+Math.exp(-sum))
    
    
    

    #END YOUR CODE
  end
  
  def adjust w
    w
  end
end

:adjust

In [18]:
### TEST ###
t31_model = LogisticRegressionModel.new
def t31_f(a: 0.0, b: 0.0)
  row = {"features" => {"a" => a, "b" => b}}
end
def t31_w(a: 0.0, b: 0.0)
  w = {"a" => a, "b" => b}
end
assert_in_delta 0.5, t31_model.predict(t31_f(), t31_w()), 1e-6
assert_in_delta 0.2689, t31_model.predict(t31_f(a:1), t31_w(a:-1)), 1e-3
assert_in_delta 1.0, t31_model.predict(t31_f(a:1, b:1000), t31_w(a:-1, b: 0.1)), 1e-3

## Question 3.2 (10 points)

Implement log loss assuming that the y label is defined as: $y \in \left\{-1, 1\right\}$. Remember that the mini-batch loss is an expectation of the $n$ examples in the mini batch.

In [19]:
class LogisticRegressionModel
  def func examples, w
    # BEGIN YOUR CODE   
    
    sum  = 0.0
    i = 0 
    obj = LinearRegressionModel.new
    adjust w

    while i< examples.length do 
      if examples[i]["label"].is_a?Numeric

        y = examples[i]["label"]
      
          #https://stats.stackexchange.com/questions/250937/which-loss-function-is-correct-for-logistic-regression#
        

           data = obj.predict(examples[i], w)
          data = 1.0/(1.0+Math.exp(y*data))
           sum += Math.log(1.0+Math.exp(-y*data))

      else
        sum += 0.0
      end
      i+=1
    end
    
    return sum/examples.length
    
    
    
    
    
    
    #END YOUR CODE
  end
  
end

:func

In [20]:
### TEST ###
t32_data = coin_dataset(1000)
t32_model = LogisticRegressionModel.new
t32_w = Hash.new {|h,k| h[k] = 0.1}
assert_in_delta 0.66, t32_model.func(t32_data["data"], t32_w), 0.2, "Expected LR.func in [460, 860]"

## Question 3.3 (10 points)

Calculate the gradient of the mini-batch log loss for each parameter $w$. This time, assume that $y \in \left\{0, 1\right\}$. Hint: This assumption should simplify the calculation.

In [21]:
class LogisticRegressionModel
  def grad examples, w
    # BEGIN YOUR CODE
    i = 0
    sum  = 0.0
    
    model= LogisticRegressionModel.new
    while i<examples.length do
      weights = Hash.new
      if w["bias"].is_a?Numeric
        weights.store("bias",w["bias"])
      else
      weights.store("bias",w[i])
      end
      sigma = model.predict(examples[i],weights)
      y = examples[i]["label"]
      sum += (-y+sigma)*examples[i]["features"]["bias"]
      i+=1
    end
    w["bias"] =  sum/examples.length
    
    return w
    
    #END YOUR CODE
  end
end

:grad

In [22]:
### TEST ###
t32_data = coin_dataset(1000)
t32_model = LogisticRegressionModel.new
t32_w = Hash.new {|h,k| h[k] = 0.1}
assert_in_delta 0.66, t32_model.func(t32_data["data"], t32_w), 0.2, "Expected LR.func in [460, 860]"
t32_g = t32_model.grad t32_data["data"], t32_w
assert_in_delta -0.26, t32_g["bias"], 0.1, "Expected LR.grad in [-0.36, -0.16]"

t32_w = Hash.new {|h,k| h[k] = 0.778}
t32_g = t32_model.grad t32_data["data"], t32_w
assert_in_delta -0.1, t32_g["bias"], 0.1, "Expected LR.grad for a closer value to be in [-0.2, -0.2]"



## Question 4.1 (6 points)

Let's train our new models on a familiar dataset, spambase. Let's run gradient descent for a few steps on this dataset. Observe that the learned weights after just gradient 2 steps are very large.

In [23]:
### Example ###
#Preview 2 lines from the Spambase dataset
spambase = read_sparse_data_from_csv "spambase"
spambase["data"].each {|r| r["features"]["bias"] = 1.0}
puts spambase["data"][0,2]

q41_model = LinearRegressionModel.new
q41_w = Hash.new {|h,k| h[k] = 0.0}
q41_w["bias"] = 1
q41_sgd = StochasticGradientDescent.new q41_model, q41_w, 0.1
2.times do
  q41_batch = spambase["data"].sample(10)
  q41_sgd.update q41_batch
end
puts q41_w

[{"features"=>{"word_freq_our"=>0.27, "word_freq_mail"=>0.83, "word_freq_you"=>0.27, "word_freq_your"=>0.27, "word_freq_font"=>8.58, "char_freq_["=>0.092, "char_freq_$"=>0.185, "char_freq_#"=>0.232, "capital_run_length_average"=>7.313, "capital_run_length_longest"=>99.0, "capital_run_length_total"=>607.0, "bias"=>1.0}, "label"=>1.0}, {"features"=>{"word_freq_your"=>0.9, "word_freq_george"=>0.9, "word_freq_data"=>0.9, "char_freq_["=>0.14, "capital_run_length_average"=>3.472, "capital_run_length_longest"=>28.0, "capital_run_length_total"=>125.0, "bias"=>1.0}, "label"=>0.0}]
{"bias"=>0.8995944084089785}


## Question 4.1 (Continued) 
We can correct this by _normalizing_ the data. A popular normalization is the z-score. For each feature, except bias, and considering only the non-zero values create a new zspambase dataset, ```zspambase```. The dataset ```zspambase``` is identical to spambase except that its features have been normalized as follows:

### $x_z = \frac{x - \mu}{\sigma}$

where $\mu$ is the mean of the $x$ value and $\sigma$ is the standard deviation. Note that you have already seen an implementation of ```mean``` and ```stdev```, so find it and add it here.

In [24]:
## Add mean and stdev here

# BEGIN YOUR CODE
def normalization 
def mean x
  # BEGIN YOUR CODE
  x.inject(0.0){|sum,i| sum + i }/x.length()  
  #END YOUR CODE
end


def stdev x
  # BEGIN YOUR CODE
  mu = mean x
  su = x.inject(0.0){|sum,i| sum + (i-mu)**2 }/(x.length()-1)
  Math.sqrt(su) 
end  




spambase = read_sparse_data_from_csv "spambase"  
dataset = spambase["data"]  
feature = spambase["features"]
data =   Array.new(feature.length) { Array.new(dataset.length){0}}

i = 0
while i<  dataset.length do
  j = 0  
  while j<feature.length do
    data[j][i]=dataset[i]["features"][feature[j]]
    j+=1
  end
  
  i+=1
end

means = Hash.new
stdev = Hash.new

j = 0  
while j<feature.length do
    m = mean data[j].reject { |e| e.to_s.empty? }
    s = stdev data[j].reject { |e| e.to_s.empty? }
    means.store(feature[j],m)
    stdev.store(feature[j],s)
    j+=1
end  
  
return [means,stdev]

end
  #END YOUR CODE



#END YOUR CODE

:normalization

In [25]:
def create_zspambase spambase
  zspambase = spambase.clone
  zspambase["data"] = spambase["data"].collect do |r|
    u = r.clone
    u["features"] = r["features"].clone
    u
  end
 
  # BEGIN YOUR CODE
  i = 0

  feature = zspambase["features"]
  means = normalization[0]
  stdev = normalization[1]
  while i<  zspambase["data"].length do
    
    j = 0  
    name = zspambase["data"][i]["features"].keys    
    while j<feature.length do
      if name.include?feature[j]
        
       
         zspambase["data"][i]["features"][feature[j]] = (zspambase["data"][i]["features"][feature[j]]-means[feature[j]])/stdev[feature[j]]
          
      end
      j+=1
    end

    i+=1
  end  
  
  
  
  
  #END YOUR CODE
  return zspambase
end

zspambase = create_zspambase spambase
zspambase["data"].first

{"features"=>{"word_freq_our"=>-0.628106690674003, "word_freq_mail"=>-0.0163998685249916, "word_freq_you"=>-1.2509960473524198, "word_freq_your"=>-0.9962817981732773, "word_freq_font"=>0.8660048920660688, "char_freq_["=>-0.19095268670254528, "char_freq_$"=>-0.162631899108401, "char_freq_#"=>-0.038069985077358884, "capital_run_length_average"=>0.06686163123580875, "capital_run_length_longest"=>0.24027347134946417, "capital_run_length_total"=>0.533869650358239, "bias"=>1.0}, "label"=>1.0}

In [26]:
### TEST ###
t41_zs = create_zspambase spambase

assert_in_delta 0.27, spambase["data"].first["features"]["word_freq_our"], 1e-5
assert_in_delta -0.628106690674003, zspambase["data"].first["features"]["word_freq_our"], 1e-5

assert_in_delta 607.0, spambase["data"].first["features"]["capital_run_length_total"], 1e-5
assert_in_delta 0.53386, zspambase["data"].first["features"]["capital_run_length_total"], 1e-5

## Question 4.2 (7 points)

Train Linear Regression for the ```zspambase``` dataset. Tune the learning rate as needed to train in one epoch. Hint: Learning rate may need to be very small. 

In [27]:
def train_zspambase_sgd(obj, w, dataset)
  i = 0
  iters = []
  losses = []
  
  #Define sgd = StochasticGradientDescent.new obj, w, lr
  # You set the learning rate, lr
  # BEGIN YOUR CODE
  lr =  0.6
  sgd =  StochasticGradientDescent.new(obj, w, lr)
  features=dataset["features"]
  i = 0

   46.times do
        iters.append(i)
        l = obj.func(dataset["data"][(i+1)*100,(i+2)*100],w)
        losses.append(l)
        sgd.update(dataset["data"][(i+1)*100,(i+2)*100])
        w = sgd.weights


        i+=1
    end

  
  
  #END YOUR CODE
  return [sgd, iters, losses]
end

:train_zspambase_sgd

In [28]:
### TEST ###
t25_model = LinearRegressionModel.new
t25_w = Hash.new {|h,k| h[k] = 0.0}
t25_w["bias"] = 1

t25_trainer, t25_iter, t25_losses = train_zspambase_sgd t25_model, t25_w, zspambase
puts t25_w

t25_cum_loss = 0.0
t25_losses.each_index {|i| t25_cum_loss += t25_losses[i]; t25_losses[i] = t25_cum_loss / (t25_iter[i] + 1)}
assert_true (t25_losses.last < 0.15), "Expected last loss value less than target"
Daru::DataFrame.new({x: t25_iter, y: t25_losses}).plot(type: :line, x: :x, y: :y) do |plot, diagram|
  plot.x_label "Batches"
  plot.y_label "Cumulative Loss"
end

{"bias"=>0.4573688595721421}


#<Nyaplot::Plot:0x000055ec46dfe610 @properties={:diagrams=>[#<Nyaplot::Diagram:0x000055ec46deb7e0 @properties={:type=>:line, :options=>{:x=>:x, :y=>:y}, :data=>"c8f4867b-b2ab-4659-98f4-8a245d236a96"}, @xrange=[0, 45], @yrange=[0.12436477258712883, 0.275]>], :options=>{:x_label=>"Batches", :y_label=>"Cumulative Loss", :zoom=>true, :width=>700, :xrange=>[0, 45], :yrange=>[0.12436477258712883, 0.275]}}>

## Question 4.3 (7 points)

Run logistic regression on the ```zspambase``` dataset, tuning the learning rate.

In [29]:
def train_zspambase_logistic_sgd(obj, w, dataset)
  i = 0
  iters = []
  losses = []
  
  #Define sgd = StochasticGradientDescent.new obj, w, lr
  # You set the learning rate, lr
  # BEGIN YOUR CODE
  lr = 10
  sgd =  StochasticGradientDescent.new(obj, w, lr)
  features=dataset["features"]
  i = 0

   20.times do
        iters.append(i)
        l = obj.func(dataset["data"][(i+1)*230,(i+2)*230],w)
        losses.append(l)
        sgd.update(dataset["data"][(i+1)*230,(i+2)*230])
        w = sgd.weights


        i+=1
    end
  
  
  
  
  #END YOUR CODE
  return [sgd, iters, losses]
end

:train_zspambase_logistic_sgd

In [30]:
### TEST ###
t43_model = LogisticRegressionModel.new
t43_w = Hash.new {|h,k| h[k] = 0.0}
t43_w["bias"] = 1

t43_trainer, t43_iter, t43_losses = train_zspambase_logistic_sgd t43_model, t43_w, zspambase
t43_cum_loss = 0.0
t43_losses.each_index {|i| t43_cum_loss += t43_losses[i]; t43_losses[i] = t43_cum_loss / (t43_iter[i] + 1)}
puts t43_w

assert_true(t43_losses.last < 0.6, "Expected last loss value < 0.6")
Daru::DataFrame.new({x: t43_iter, y: t43_losses}).plot(type: :line, x: :x, y: :y) do |plot, diagram|
  plot.x_label "Batches"
  plot.y_label "Cumulative Loss"
end

{"bias"=>0.6380960833510414}


#<Nyaplot::Plot:0x000055ec46de84c8 @properties={:diagrams=>[#<Nyaplot::Diagram:0x000055ec46de0020 @properties={:type=>:line, :options=>{:x=>:x, :y=>:y}, :data=>"973256f5-715d-45d8-ad08-f41b762d0cec"}, @xrange=[0, 19], @yrange=[0.5958822791297451, 0.6424190602284295]>], :options=>{:x_label=>"Batches", :y_label=>"Cumulative Loss", :zoom=>true, :width=>700, :xrange=>[0, 19], :yrange=>[0.5958822791297451, 0.6424190602284295]}}>